# PART3: Multilabel-Classification using scikit-learn SVM

## Load Data

In [1]:
import numpy as np
import pandas as pd 

data_root = '/media/sf_Yelp/input/'

train_photos = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv', index_col='photo_id')

train_df = pd.read_csv(data_root+"train_biz_fc7features.csv")
test_df  = pd.read_csv(data_root+"test_biz_fc7features.csv")

y_train = train_df['label'].values
X_train = train_df['feature vector'].values
X_test = test_df['feature vector'].values

def convert_label_to_array(str_label):
    str_label = str_label[1:-1]
    str_label = str_label.split(',')
    return [int(x) for x in str_label if len(x)>0]

def convert_feature_to_vector(str_feature):
    str_feature = str_feature[1:-1]
    str_feature = str_feature.split(',')
    return [float(x) for x in str_feature]

y_train = np.array([convert_label_to_array(y) for y in train_df['label']])
X_train = np.array([convert_feature_to_vector(x) for x in train_df['feature vector']])
X_test = np.array([convert_feature_to_vector(x) for x in test_df['feature vector']])


In [2]:
print "X_train: ", X_train.shape
print "y_train: ", y_train.shape
print "X_test: ", X_test.shape
print "train_df:"
train_df[0:5]

X_train:  (1996, 4096)
y_train:  (1996,)
X_test:  (10000, 4096)
train_df:


,business,label,feature vector
0,1000.0,"(1, 2, 3, 4, 5, 6, 7)","[0.20032024, 0.44084537, 0.23249489, 0.3600976..."
1,1001.0,"(0, 1, 6, 8)","[0.0013769998, 0.59398097, 0.55060995, 0.18394..."
2,100.0,"(1, 2, 4, 5, 6, 7)","[0.11435749, 0.033177156, 0.12572332, 0.539482..."
3,1006.0,"(1, 2, 4, 5, 6)","[0.075851507, 0.052600037, 0.059594199, 0.7067..."
4,1010.0,"(0, 6, 8)","[0.39024171, 0.28424361, 0.0, 0.1655795, 0.460..."


## Train a SVM using 80% training data, and assess performance(F1-score)

In [3]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

import time
t=time.time()

mlb = MultiLabelBinarizer()
y_ptrain= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = train_test_split(X_train, y_ptrain, test_size=.2,random_state=random_state)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
classifier.fit(X_ptrain, y_ptrain)

y_ppredict = classifier.predict(X_ptest)

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

Time passed:  267.0 sec


In [4]:
print "Samples of predicted labels (in binary matrix):\n", y_ppredict[0:3]
print "\nSamples of predicted labels:\n", mlb.inverse_transform(y_ppredict[0:3])

Samples of predicted labels (in binary matrix):
[[0 0 0 0 0 1 1 0 1]
 [0 0 1 0 0 0 1 0 1]
 [0 1 1 0 0 1 1 0 1]]

Samples of predicted labels:
[(5, 6, 8), (2, 6, 8), (1, 2, 5, 6, 8)]


In [5]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_ppredict, axis=0), len(y_ppredict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_ppredict) 
statistics

,attribuite 0,attribuite 1,attribuite 2,attribuite 3,attribuite 4,attribuite 5,attribuite 6,attribuite 7,attribuite 8,num_biz
biz count,133,199,205,208,112,246,266,113,242,400
biz ratio,33%,50%,51%,52%,28%,62%,66%,28%,60%,100%


In [6]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)

F1 score:  0.796719390744
Individual Class F1 score:  [ 0.66911765  0.78880407  0.85138539  0.61964736  0.75925926  0.8266129
  0.91557223  0.73777778  0.85773196]


## Re-Train a SVM using all training data, and make predictions on test set

In [3]:
## Uncomment if skip previous train
from sklearn import svm
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import time

t = time.time()

mlb = MultiLabelBinarizer()
y_train= mlb.fit_transform(y_train)  #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=True))
classifier.fit(X_train, y_train)

y_predict = classifier.predict(X_test)

#print list(mlb.classes_)
y_predict_label = mlb.inverse_transform(y_predict) #Convert binary matrix back to labels

print "Time passed: ", "{0:.1f}".format(time.time()-t), "sec"

ValueError: X.shape[0] should be equal to X.shape[1]

In [ ]:
X_test.shape

In [ ]:
test_data_frame  = pd.read_csv(data_root+"test_biz_fc7features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_data_frame)):
    biz = test_data_frame.loc[i]['business']
    label = y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(biz), label]

with open(data_root+"../submissions/svc_rbf_submission_fc7.csv",'w') as f:
    df.to_csv(f, index=False)    
    

In [ ]:
statistics = pd.DataFrame(columns=[ "attribuite "+str(i) for i in range(9)]+['num_biz'], index = ["biz count", "biz ratio"])
statistics.loc["biz count"] = np.append(np.sum(y_predict, axis=0), len(y_predict))
pd.options.display.float_format = '{:.0f}%'.format
statistics.loc["biz ratio"] = statistics.loc["biz count"]*100/len(y_predict) 
statistics

In [12]:
#LB score: 0.76437 (use fc7 layer)
#LB score: 0.73053 (use prob layer)